In [1]:
import sys
import os

# moce two directories up and make that current directory
os.chdir("../..")
# print current directory
print(os.getcwd())

/Users/chinmayshrivastava/Documents/GitHub/kinara-rfp


In [2]:
from govrfp.dynamicfs.Selector import Selector
from govrfp.rfpparser.RFPParser import RFPParser
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
rfp = RFPParser(
    pdf_path="tests/mar-21/egsearch.pdf",
    verbose=True
)

/Users/chinmayshrivastava/Documents/GitHub/kinara-rfp/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [3]:
chunks = rfp.chunks()

In [4]:
prompt_template = """You will be given excerpts from an RFP application. Your task is to extract all compliance requirements that the applicant needs to meet while drafting the proposal. Break down complex requirements into simpler, standalone ones, rephrase them into an actionable form, and identify all requirements even if they are embedded within a sentence or phrase. If no requirements are found, return 'None'. Ignore any information that is not a compliance requirement.

Example 1: 

Excerpt:
Deadline: 08/31/2023 1700 ET
Offer due date: 08/31/2023 1700 ET
Submit invoices to address shown in block 18a 

Response:
Submit the application by August 31, 2023, 5:00 PM ET
Send invoices to the address in block 18a

Example 2: 
Excerpt:
Any deviation or delay upon the agreed upon delivery time will be recorded and influence the contractor closing assessment made by the COR Offeror must provide description of service(s) or product(s). Number and Names of Sub-contractors, if any 

Response:
Ensure no deviation or delay from the agreed delivery time
Provide a description of the service(s) or product(s)
List the number and names of any sub-contractors, if any

----------------
For the following excerpt, provide the compliance requirements:

Excerpt:
{excerpt}

Response:"""

In [5]:
import tqdm
compliances = []
for chunk in tqdm.tqdm(chunks):
    prompt = prompt_template.format(excerpt=chunk['text'])
    response = llm.complete(prompt).text
    compliances.append(response)

  0%|          | 0/77 [00:00<?, ?it/s]

100%|██████████| 77/77 [03:00<00:00,  2.34s/it]


In [7]:
# save the compliances
with open("tests/mar-21/02-compliances.txt", "w") as f:
    for comp in compliances:
        f.write(comp + "\n")